# Entendimiento de datos

In [1]:
# Importaciones
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split

## 1. Utils

In [2]:
# Variables de configuración
SEED = 9603

MUESTRA = 0.125
# 0.3678028341415188 / 500K
# 0.1218641130361094 / 165K
# 0.2907504151915929

# Validación mediante K-Iteraciones
# k = 10
TEST_SIZE = 0.1

files_paths = [
  "Monday-WorkingHours.csv",
  "Tuesday-WorkingHours.csv",
  "Wednesday-WorkingHours.csv",
  "Thursday-WorkingHours.csv",
  "Friday-WorkingHours.csv",
]

selected_files_paths = [
  # "Monday-WorkingHours.csv",
  "Tuesday-WorkingHours.csv",
  "Wednesday-WorkingHours.csv",
  # "Thursday-WorkingHours.csv",
  "Friday-WorkingHours.csv",
]

output_path = '../3_preparacion_de_datos/DB/dataset.csv'

In [3]:
# getFrequency(): Función para obtener la distribución de frecuencias de la columna label
def getFrequency(df : pd.DataFrame, caracteristica) :
  frecuencia = df[caracteristica].value_counts()
  porcentaje = df[caracteristica].value_counts(normalize=True) * 100

  tabla_frecuencia = pd.DataFrame({
    "Frecuencia": frecuencia,
    "Frecuencia(%)": porcentaje
  })
  
  tabla_frecuencia["Frecuencia(%)"].round(2)
  
  print(tabla_frecuencia)
  print(df.shape)

In [4]:
# generateCorrMatrix(): Función para generar matriz de correlación
def generateCorrMatrix(data, save_path="", title="Matriz de correlación", method="pearson", width=32, height=24, decimals=2, abs = False, save=True, aplicar_umbral=False, umbral=0.5) :
  # Generación de matriz de correlación
  matriz_correlacion = data.corr(method=method).round(decimals=decimals)

  if abs :
    matriz_correlacion = matriz_correlacion.abs()

  if aplicar_umbral : 
    matriz_correlacion = matriz_correlacion.where(np.abs(matriz_correlacion) >= umbral, 0)

  # Generación de mapa de calor para visualizar los datos
  plt.figure(figsize=(width, height))

  sns.heatmap(matriz_correlacion, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
  plt.title(title)
  plt.tight_layout()
  if save :
    plt.savefig(save_path)
  plt.show()
  return matriz_correlacion

## 2. Carga de datos

In [5]:
# Existen ataques que tienen muy poca representación en la base de datos
days = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes']
for i in range(len(days)) : 
  df_per_day = pd.read_csv(f'DB/{files_paths[i]}')
  print(f'\n{days[i]}\n')
  getFrequency(df=df_per_day, caracteristica='Label')


Lunes

        Frecuencia  Frecuencia(%)
Label                            
BENIGN      371749          100.0
(371749, 84)

Martes

                         Frecuencia  Frecuencia(%)
Label                                             
BENIGN                       315031      97.834803
FTP-Patator                    3973       1.233839
SSH-Patator                    2980       0.925457
FTP-Patator - Attempted          11       0.003416
SSH-Patator - Attempted           8       0.002484
(322003, 84)

Miércoles

                              Frecuencia  Frecuencia(%)
Label                                                  
BENIGN                            319216      64.257145
DoS Hulk                          158469      31.899295
DoS GoldenEye                       7567       1.523213
DoS slowloris                       4001       0.805388
DoS Slowhttptest - Attempted        3369       0.678169
DoS Slowhttptest                    1742       0.350659
DoS slowloris - Attempted           17

In [6]:
# Se excluiran de la base de datos los días Lunes y Jueves
# Lunes debido a que solo representan tráfico normal
# Jueves debido a que hay ataques que no tienen suficiente representación 
df = pd.concat(
  [pd.read_csv(f"DB/{file_path}") for file_path in selected_files_paths]
)
getFrequency(df=df, caracteristica="Label")

                              Frecuencia  Frecuencia(%)
Label                                                  
BENIGN                            925680      67.731180
PortScan                          159151      11.644937
DoS Hulk                          158469      11.595035
DDoS                               95123       6.960065
DoS GoldenEye                       7567       0.553671
DoS slowloris                       4001       0.292750
FTP-Patator                         3973       0.290701
DoS Slowhttptest - Attempted        3369       0.246507
SSH-Patator                         2980       0.218044
DoS Slowhttptest                    1742       0.127461
DoS slowloris - Attempted           1731       0.126656
Bot - Attempted                     1470       0.107559
Bot                                  738       0.053999
DoS Hulk - Attempted                 593       0.043389
DoS GoldenEye - Attempted             80       0.005854
FTP-Patator - Attempted               11       0

In [7]:
# Se eliminarán los ciberataques Infiltration y Heartbleed y registros con sufijo "- Attempted"
# los ataques Infiltration y Heartbleed debido a que no tienen suficiente representación en la base de datos 
# y los ataques con sufijo "- Attempted" debido a que no representan ciberataques
df = df[~df["Label"].isin([
  'DoS Slowhttptest - Attempted',
  'DoS slowloris - Attempted',
  'Bot - Attempted',
  'DoS Hulk - Attempted',
  'DoS GoldenEye - Attempted',
  'Infiltration - Attempted',
  'FTP-Patator - Attempted',
  'SSH-Patator - Attempted',
  'Infiltration',
  'Heartbleed',
])]
getFrequency(df=df, caracteristica="Label")

                  Frecuencia  Frecuencia(%)
Label                                      
BENIGN                925680      68.093546
PortScan              159151      11.707238
DoS Hulk              158469      11.657069
DDoS                   95123       6.997302
DoS GoldenEye           7567       0.556633
DoS slowloris           4001       0.294316
FTP-Patator             3973       0.292256
SSH-Patator             2980       0.219210
DoS Slowhttptest        1742       0.128143
Bot                      738       0.054288
(1359424, 84)


## 3. Descripción de datos

In [8]:
# Extracción de muestra para el proyecto
df_vector, df_temp = train_test_split(
  df, 
  stratify=df["Label"], 
  test_size=1-MUESTRA, 
  random_state=SEED
)
getFrequency(df=df_vector, caracteristica="Label")

                  Frecuencia  Frecuencia(%)
Label                                      
BENIGN                115710      68.093546
PortScan               19894      11.707311
DoS Hulk               19809      11.657290
DDoS                   11890       6.997081
DoS GoldenEye            946       0.556706
DoS slowloris            500       0.294242
FTP-Patator              497       0.292477
SSH-Patator              372       0.218916
DoS Slowhttptest         218       0.128290
Bot                       92       0.054141
(169928, 84)


In [9]:
# Guardando dataset
df_vector.to_csv(output_path, index=False)
print(f'Muestra para el proyecto guardada en "{output_path}"')

Muestra para el proyecto guardada en "../3_preparacion_de_datos/DB/dataset.csv"


## 4. Exploración de datos

In [10]:
# Tipos de características en el conjunto de datos CICIDS2017(Actualización 2021)
print(df_vector.dtypes.head(42))
print(df_vector.dtypes.tail(42))

Flow ID                        object
Src IP                         object
Src Port                        int64
Dst IP                         object
Dst Port                        int64
Protocol                        int64
Timestamp                      object
Flow Duration                   int64
Total Fwd Packet                int64
Total Bwd packets               int64
Total Length of Fwd Packet    float64
Total Length of Bwd Packet    float64
Fwd Packet Length Max         float64
Fwd Packet Length Min         float64
Fwd Packet Length Mean        float64
Fwd Packet Length Std         float64
Bwd Packet Length Max         float64
Bwd Packet Length Min         float64
Bwd Packet Length Mean        float64
Bwd Packet Length Std         float64
Flow Bytes/s                  float64
Flow Packets/s                float64
Flow IAT Mean                 float64
Flow IAT Std                  float64
Flow IAT Max                  float64
Flow IAT Min                  float64
Fwd IAT Tota

## 5. Verificación de calidad de datos

In [11]:
# Visualización de valores nulos
null_chars_list = df_vector.isna().sum()
null_chars_list = null_chars_list[null_chars_list > 0]
null_chars_list

Flow Bytes/s     31
Flow IAT Mean    13
Flow IAT Std     13
Flow IAT Max     13
Flow IAT Min     13
dtype: int64

In [12]:
# Visualización de valores infinitos
inf_chars_list = np.isinf(df_vector.select_dtypes(include=[np.float64, np.int64])).sum()
inf_chars_list = inf_chars_list[inf_chars_list > 0]
inf_chars_list

Flow Bytes/s      13
Flow Packets/s    44
dtype: int64

In [13]:
# Descripción de características

# las características numéricas están relacionadas con características presentes en cada grupo
caracteristicas_numericas = [
  'Flow Duration', 'Total Fwd Packet', 'Total Bwd packets', 
  'Total Length of Fwd Packet', 'Total Length of Bwd Packet', 
  'Fwd Packet Length Max', 'Fwd Packet Length Min', 
  'Fwd Packet Length Mean', 'Fwd Packet Length Std', 
  'Bwd Packet Length Max', 'Bwd Packet Length Min', 
  'Bwd Packet Length Mean', 'Bwd Packet Length Std', 
  'Flow Bytes/s', 'Flow Packets/s',
  'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
  'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
  'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
  'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
  'Fwd Header Length',
  'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
  'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
  'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
  'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
  'CWR Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
  'Average Packet Size', 'Fwd Segment Size Avg', 'Bwd Segment Size Avg',
  'Fwd Bytes/Bulk Avg', 'Fwd Packet/Bulk Avg', 'Fwd Bulk Rate Avg',
  'Bwd Bytes/Bulk Avg', 'Bwd Packet/Bulk Avg', 'Bwd Bulk Rate Avg',
  'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets',
  'Subflow Bwd Bytes', 'FWD Init Win Bytes', 'Bwd Init Win Bytes',
  'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
  'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
  'Idle Min'
]

caracteristicas_var_null = ['Flow Bytes/s','Flow IAT Mean','Flow IAT Std','Flow IAT Max','Flow IAT Min']

caracteristicas_var_max_inf = ['Flow Bytes/s', 'Flow Packets/s']

# Características que no registraron información para análisis
caracteristicas_no_utiles = [
  'Flow ID', 'Src IP', 'Src Port', 
  'Dst IP', 'Timestamp', 'Fwd URG Flags', 
  'Bwd URG Flags', 'URG Flag Count'
]

caracteristicas_nominales = ['Dst Port', 'Protocol']

caracteristica_objetivo = "Label"

print(f'total de características : {len(df_vector.columns)}\n{list(df_vector.columns)}\n')

print(f'caracteristicas_numericas : {len(caracteristicas_numericas)}\n{caracteristicas_numericas}\n')

print(f'caracteristicas_var_null : {len(caracteristicas_var_null)}\n{caracteristicas_var_null}\n')

print(f'caracteristicas_var_max_inf : {len(caracteristicas_var_max_inf)}\n{caracteristicas_var_max_inf}\n')

print(f'caracteristicas_no_utiles : {len(caracteristicas_no_utiles)}\n{caracteristicas_no_utiles}\n')

print(f'caracteristicas_nominales : {len(caracteristicas_nominales)}\n{caracteristicas_nominales}\n')

print(f'caracteristica_objetivo : {len([caracteristica_objetivo])}\n{[caracteristica_objetivo]}')


total de características : 84
['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Total Fwd Packet', 'Total Bwd packets', 'Total Length of Fwd Packet', 'Total Length of Bwd Packet', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH 